In [1]:
# Add graphviz to path
import os

os.environ["PATH"] = os.environ["PATH"] + ":/opt/homebrew/bin"

# Imports
from vaxflux.curves import LogisticIncidenceCurve
from vaxflux.uptake import DateRange, PooledCovariate, SeasonRange, SeasonalUptakeModel

WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
logistic_curve = LogisticIncidenceCurve()
logistic_curve

In [3]:
logistic_curve.parameters

('m', 'r', 's')

In [4]:
model = SeasonalUptakeModel(
    logistic_curve,
    [
        PooledCovariate(
            parameter="m",
            covariate=None,
            distribution="Normal",
            distribution_kwargs={"mean": 0.5, "sd": 0.1},
        )
    ],
    season_ranges=[
        SeasonRange(season="2022/2023", start_date="2022-09-01", end_date="2023-08-31"),
        SeasonRange(season="2023/2024", start_date="2023-09-01", end_date="2024-08-31"),
    ],
    date_ranges=[
        DateRange(
            season="2022/2023",
            start_date="2022-09-01",
            end_date="2022-09-30",
            report_date="2022-10-01",
        ),
    ],
    name="Test Model",
)